In [18]:
# https://www.kaggle.com/datasets/christofferms/pokemon-with-stats-and-image/data

import pandas as pd

data = pd.read_csv("pokedex.csv")

data = data.iloc[:, 3:]
data = data.rename(columns={"Type 1": "1. Element", "Type 2": "2. Element"})

data

,1. Element,2. Element,Total,HP,Attack,Defense,SP. Atk.,SP. Def,Speed
0,Grass,Poison,318,45,49,49,65,65,45
1,Grass,Poison,405,60,62,63,80,80,60
2,Grass,Poison,525,80,82,83,100,100,80
3,Grass,Poison,625,80,100,123,122,120,80
4,Fire,NaN,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...
1210,Steel,Psychic,590,90,72,100,122,108,98
1211,Normal,NaN,450,90,65,85,65,85,60
1212,Normal,NaN,600,95,95,110,105,110,85
1213,Normal,NaN,700,160,105,110,130,110,85


Snažil jsem se najít si vlastní dataset, který by mě zajímal a nakonec jsem nečekaně skončil u pokémonů, kde jsem si řekl, že by to mohl být zajímavý dataset na analýzu. Dataset obsahuje pokémony z 1-9 řady pokémonů.

Konkrétně u tohoto datasetu máme záznam o 1215 pokémonech s tím, že známe typy pokémonů, jejich podrobné statistiky, takže budeme schopni určit trend pokémonů, kterých je nejvíce, kteří jsou nejsilnější a poté utvořit shlukování, které pravděpodobně bude shlukovat podle typů pokémonů.

U datasetu máme celkově 12 sloupců, kde rovnou dropnu 1. sloupec s "Image", protože pro mé účely není vůbec potřeba tento sloupec s odkazem na obrázek k danému pokémonovi mít. Taktéž "Index" sloupec hned následující dropnu, protože číselné označení pokémonů pro mě nemá žádný význam. Taktéž název pokémona mě moc nezajímá, protože mi nijak nebude určovat nic v datasetu a nejsem schopný k němu vázat žádné závislosti.

Po očištění datasetu nám tedy zbývá 9 sloupců.

Type 1 - 1. typ (element) pokémona

Type 2 - _.._ (může být NaN)

Total - součet všech hodnot 

HP - životy pokémona

Attack - Poškození pokémona

Defense - Obrana pokémona

SP. Atk. - Poškození pomocí schopnosti

SP. Def - Obrana proti schopnostem

Speed - Rychlost pohybu pokémona

Sloupec (Type 1, Type 2) si dovolím přejmenovat na 1. Element a 2. Element.

In [19]:
data.isna().sum()

1. Element      0
2. Element    546
Total           0
HP              0
Attack          0
Defense         0
SP. Atk.        0
SP. Def         0
Speed           0
dtype: int64

Jak můžeme vidět, všechny sloupce až na 2. Element mají hodnoty. Jelikož je pro nás 2. sloupec důležitý a nemůžeme se ho zbavit, vyplním veškeré chybějí hodnoty třeba stringem "null" a později ho můžu vyfiltrovat v grafech a různých ukázkách a počtech.

In [20]:
data.fillna("None", inplace=True)

,1. Element,2. Element,Total,HP,Attack,Defense,SP. Atk.,SP. Def,Speed
0,Grass,Poison,318,45,49,49,65,65,45
1,Grass,Poison,405,60,62,63,80,80,60
2,Grass,Poison,525,80,82,83,100,100,80
3,Grass,Poison,625,80,100,123,122,120,80
4,Fire,None,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...
1210,Steel,Psychic,590,90,72,100,122,108,98
1211,Normal,None,450,90,65,85,65,85,60
1212,Normal,None,600,95,95,110,105,110,85
1213,Normal,None,700,160,105,110,130,110,85


Nyní je všechno nahrazené za "None" hodnoty.